In [85]:
import pandas as pd       
import numpy as np      
#data visualisation
import seaborn as sns     
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import warnings; warnings.simplefilter('ignore')


In [86]:
#load data
df = pd.read_csv('nyc_taxi_trip_duration.csv')
#see sample data
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [87]:
df.shape

(729322, 11)

In [88]:
df.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,729322.000000,729322.000000,729322.000000,729322.000000,729322.000000,729322.000000,7.293220e+05
mean,1.535403,1.662055,-73.973513,40.750919,-73.973422,40.751775,9.522291e+02
std,0.498745,1.312446,0.069754,0.033594,0.069588,0.036037,3.864626e+03
min,1.000000,0.000000,-121.933342,34.712234,-121.933304,32.181141,1.000000e+00
25%,1.000000,1.000000,-73.991859,40.737335,-73.991318,40.735931,3.970000e+02
50%,2.000000,1.000000,-73.981758,40.754070,-73.979759,40.754509,6.630000e+02
75%,2.000000,2.000000,-73.967361,40.768314,-73.963036,40.769741,1.075000e+03
max,2.000000,9.000000,-65.897385,51.881084,-65.897385,43.921028,1.939736e+06


In [89]:
# making more feactures for better prediction
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime']=pd.to_datetime(df['dropoff_datetime'])

df['pickup_day']=df['pickup_datetime'].dt.day_name()
df['dropoff_day']=df['dropoff_datetime'].dt.day_name()
df['pickup_day_no']=df['pickup_datetime'].dt.weekday
df['dropoff_day_no']=df['dropoff_datetime'].dt.weekday
df['pickup_hour']=df['pickup_datetime'].dt.hour
df['dropoff_hour']=df['dropoff_datetime'].dt.hour
df['pickup_month']=df['pickup_datetime'].dt.month
df['dropoff_month']=df['dropoff_datetime'].dt.month

In [90]:
# making more feactures for better prediction by extracting time of day from the dates
def time_of_day(x):
    if x in range(6,12):
        return 'Morning'
    elif x in range(12,16):
        return 'Afternoon'
    elif x in range(16,22):
        return 'Evening'
    else:
        return 'Late night'

In [91]:
# applying time of day function to pickup and dropoff hour
df["pickup_timeofday"]=df["pickup_hour"].apply(time_of_day)
df["dropoff_timeofday"]=df["dropoff_hour"].apply(time_of_day)

In [92]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from math import radians, sin, cos, sqrt, atan2

In [93]:
# using haversine distance formula to calculate distance between pickup and dropoff points
def haversine_distance(lon1, lat1, lon2, lat2):
    # Convert degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    # Earth radius in kilometers (6371 km)
    r = 6371
    distance = r * c
    return distance

In [94]:
# applying cal_distance function to pickup and dropoff latitude and longitude
df["distance"] = df.apply(lambda x: haversine_distance(x["pickup_longitude"],x["pickup_latitude"],x["dropoff_longitude"],x["dropoff_latitude"] ), axis=1)


In [95]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,dropoff_day,pickup_day_no,dropoff_day_no,pickup_hour,dropoff_hour,pickup_month,dropoff_month,pickup_timeofday,dropoff_timeofday,distance
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,...,Monday,0,0,16,16,2,2,Evening,Evening,1.199073
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,...,Friday,4,4,23,23,3,3,Late night,Late night,4.129111
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,...,Sunday,6,6,17,18,2,2,Evening,Evening,7.250753
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,...,Tuesday,1,1,9,10,1,1,Morning,Morning,2.361097
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,...,Wednesday,2,2,6,6,2,2,Morning,Morning,4.328534


In [96]:
# create a prediction model using linear regression and predict the trip duration for the test data set
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler



In [97]:
# droping the columns that are not needed
df.drop(['id','pickup_datetime','dropoff_datetime','pickup_day','dropoff_day','pickup_timeofday','dropoff_timeofday','store_and_fwd_flag'],axis=1,inplace=True)
# splitting the data into train and test
X_train,X_test,y_train,y_test=train_test_split(df.drop('trip_duration',axis=1),df['trip_duration'],test_size=0.3,random_state=42)
df.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,pickup_day_no,dropoff_day_no,pickup_hour,dropoff_hour,pickup_month,dropoff_month,distance
0,2,1,-73.953918,40.778873,-73.963875,40.771164,400,0,0,16,16,2,2,1.199073
1,1,2,-73.988312,40.731743,-73.994751,40.694931,1100,4,4,23,23,3,3,4.129111
2,2,2,-73.997314,40.721458,-73.948029,40.774918,1635,6,6,17,18,2,2,7.250753
3,2,6,-73.961670,40.759720,-73.956779,40.780628,1141,1,1,9,10,1,1,2.361097
4,1,1,-74.017120,40.708469,-73.988182,40.740631,848,2,2,6,6,2,2,4.328534


In [98]:
# scaling the data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [99]:
# creating a linear regression model
lr=LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [100]:
# predicting the trip duration for the test data set
y_pred=lr.predict(X_test)

In [101]:
# calculating the mean squared error
mean_squared_error(y_test,y_pred)

26231352.971600868

In [102]:
# calculating the root mean squared error
np.sqrt(mean_squared_error(y_test,y_pred))

5121.655296054281

In [103]:
# show results of the prediction
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results.head(25)


,Actual,Predicted
469114,657,877.538173
694852,195,786.773220
696324,840,1298.697935
356496,398,584.025539
645318,682,766.672335
498463,758,932.428011
296444,197,606.206720
191793,873,965.638369
531779,1019,729.059285
476548,591,1088.503884


In [104]:
# applying log transformation to the trip duration to make it more normally distributed
df['trip_duration_log'] = np.log(df['trip_duration'].values + 1)
df.head()


,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,pickup_day_no,dropoff_day_no,pickup_hour,dropoff_hour,pickup_month,dropoff_month,distance,trip_duration_log
0,2,1,-73.953918,40.778873,-73.963875,40.771164,400,0,0,16,16,2,2,1.199073,5.993961
1,1,2,-73.988312,40.731743,-73.994751,40.694931,1100,4,4,23,23,3,3,4.129111,7.003974
2,2,2,-73.997314,40.721458,-73.948029,40.774918,1635,6,6,17,18,2,2,7.250753,7.400010
3,2,6,-73.961670,40.759720,-73.956779,40.780628,1141,1,1,9,10,1,1,2.361097,7.040536
4,1,1,-74.017120,40.708469,-73.988182,40.740631,848,2,2,6,6,2,2,4.328534,6.744059


In [105]:
# splitting the data into train and test
X_train,X_test,y_train,y_test=train_test_split(df.drop(['trip_duration','trip_duration_log'],axis=1),df['trip_duration_log'],test_size=0.3,random_state=42)

# scaling the data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# creating a linear regression model
lr=LinearRegression()
lr.fit(X_train,y_train)

# predicting the trip duration for the test data set
y_pred=lr.predict(X_test)

# calculating the mean squared error
mean_squared_error(y_test,y_pred)

0.41197536268810686

In [106]:
# results of the prediction for the log transformed trip duration
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results.head(25)

,Actual,Predicted
469114,6.489205,6.309897
694852,5.278115,6.213459
696324,6.734592,6.897028
356496,5.988961,6.205353
645318,6.526495,6.361523
498463,6.632002,6.358218
296444,5.288267,6.066937
191793,6.773080,6.391385
531779,6.927558,6.351482
476548,6.383507,6.538856
